In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount = True)

Mounted at /content/drive


In [ ]:
%cd drive/Shareddrives/'NEXUS-PARSEC - TCCs'/ProjetoRaf-Car-Igor/data/indicadores_IDH_Nexus

/content/drive/Shareddrives/NEXUS-PARSEC - TCCs/ProjetoRaf-Car-Igor/data/indicadores_IDH_Nexus


In [ ]:
!pip install geopandas

In [ ]:
import geopandas

In [ ]:
geo = geopandas.read_file('setores_shapefile/ac_setores_censitarios/12SEE250GC_SIR.shp')
geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID          900 non-null    int64   
 1   CD_GEOCODI  900 non-null    object  
 2   TIPO        900 non-null    object  
 3   CD_GEOCODB  0 non-null      object  
 4   NM_BAIRRO   0 non-null      object  
 5   CD_GEOCODS  900 non-null    object  
 6   NM_SUBDIST  0 non-null      object  
 7   CD_GEOCODD  900 non-null    object  
 8   NM_DISTRIT  900 non-null    object  
 9   CD_GEOCODM  900 non-null    object  
 10  NM_MUNICIP  900 non-null    object  
 11  NM_MICRO    900 non-null    object  
 12  NM_MESO     900 non-null    object  
 13  geometry    900 non-null    geometry
dtypes: geometry(1), int64(1), object(12)
memory usage: 98.6+ KB


In [ ]:
from os import listdir
import glob
import pandas

In [ ]:
shapefiles_list = glob.glob('setores_shapefile/*/*.shp')
geo = geopandas.read_file(shapefiles_list[0])

In [ ]:
for file in shapefiles_list[1:]:
  geoaux = geopandas.read_file(file)
  geo = pandas.concat([geo,geoaux],ignore_index = True)

In [ ]:
geo = geo.rename(columns ={'CD_GEOCODI':'Cod_setor','CD_GEOCODM' : 'Cod_municipio'})
geo = geo.astype({'Cod_municipio' : 'int64'})
geo = geo.drop(columns = ['ID','CD_GEOCODB','NM_BAIRRO','CD_GEOCODS','NM_SUBDIST','CD_GEOCODD'])
geo = geo.drop(columns = ['NM_DISTRIT','NM_MUNICIP','NM_MICRO','NM_MESO','ID1'])
geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 316574 entries, 0 to 316573
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Cod_setor      316574 non-null  object  
 1   TIPO           316574 non-null  object  
 2   Cod_municipio  316574 non-null  int64   
 3   geometry       316574 non-null  geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 9.7+ MB


In [ ]:
shapefiles_list2 = glob.glob('processed/*.csv')
geo2 = geopandas.read_file(shapefiles_list2[0])

In [ ]:
for file in shapefiles_list2[1:]:
  geoaux = geopandas.read_file(file)
  geo2 = pandas.concat([geo2,geoaux],ignore_index = True)

In [ ]:
geo2 = geo2.drop(columns = ['geometry','field_1'])

geo2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 110214 entries, 0 to 110213
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Cod_setor   110214 non-null  object
 1   IDHL-Setor  110214 non-null  object
 2   AlfabIDHM   110214 non-null  object
 3   RendaIDHM   110214 non-null  object
dtypes: object(4)
memory usage: 3.4+ MB


In [ ]:
geototal = geo2.merge(geo,on = 'Cod_setor')

In [ ]:
geototal.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 110214 entries, 0 to 110213
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Cod_setor      110214 non-null  object  
 1   IDHL-Setor     110214 non-null  object  
 2   AlfabIDHM      110214 non-null  object  
 3   RendaIDHM      110214 non-null  object  
 4   TIPO           110214 non-null  object  
 5   Cod_municipio  110214 non-null  int64   
 6   geometry       110214 non-null  geometry
dtypes: geometry(1), int64(1), object(5)
memory usage: 6.7+ MB


In [ ]:
geototal['centroid'] = geototal['geometry'].centroid
# #Extract lat and lon from the centerpoint
geototal["lat"] = geototal['centroid'].map(lambda p: p.x)
geototal["lon"] = geototal['centroid'].map(lambda p: p.y)
geototal['area'] = geototal['geometry'].area*1.25e4

In [ ]:
import math
cod_municipio = geototal['Cod_municipio']
cod_estado = (cod_municipio/100000)
cod_estado = cod_estado.astype('int64')
geototal['Cod_estado'] = (cod_estado)

In [ ]:
geototal.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 110214 entries, 0 to 110213
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Cod_setor      110214 non-null  object  
 1   IDHL-Setor     110214 non-null  object  
 2   AlfabIDHM      110214 non-null  object  
 3   RendaIDHM      110214 non-null  object  
 4   TIPO           110214 non-null  object  
 5   Cod_municipio  110214 non-null  int64   
 6   geometry       110214 non-null  geometry
 7   centroid       110214 non-null  geometry
 8   lat            110214 non-null  float64 
 9   lon            110214 non-null  float64 
 10  area           110214 non-null  float64 
 11  Cod_estado     110214 non-null  int64   
dtypes: float64(3), geometry(2), int64(2), object(5)
memory usage: 10.9+ MB


In [ ]:
geototal = geototal.rename(columns = {'IDHL-Setor':'longevity','AlfabIDHM':'literacy','RendaIDHM':'income'})
geototal = geototal.assign(year = 2010)
columns_to_show=['year','Cod_setor','lat', 'lon','TIPO','Cod_municipio','Cod_estado','income','longevity','literacy','centroid','geometry','area']
geototal = geototal[columns_to_show]


In [ ]:
geototal.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 110214 entries, 0 to 110213
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   year           110214 non-null  int64   
 1   Cod_setor      110214 non-null  object  
 2   lat            110214 non-null  float64 
 3   lon            110214 non-null  float64 
 4   TIPO           110214 non-null  object  
 5   Cod_municipio  110214 non-null  int64   
 6   Cod_estado     110214 non-null  int64   
 7   income         110214 non-null  object  
 8   longevity      110214 non-null  object  
 9   literacy       110214 non-null  object  
 10  centroid       110214 non-null  geometry
 11  geometry       110214 non-null  geometry
 12  area           110214 non-null  float64 
dtypes: float64(3), geometry(2), int64(3), object(5)
memory usage: 11.8+ MB


In [ ]:
geototal.head()

,year,Cod_setor,lat,lon,TIPO,Cod_municipio,Cod_estado,income,longevity,literacy,centroid,geometry,area
0,2010,210010505000001,-43.321169,-4.135545,URBANO,2100105,21,0.34030523371478993,0.7333333333333333,0.7511683506133089,POINT (-43.32117 -4.13555),"POLYGON ((-43.32326 -4.13234, -43.32125 -4.132...",0.448939
1,2010,210010505000002,-43.319907,-4.131433,URBANO,2100105,21,0.3484755978662226,0.75,0.8220312377364415,POINT (-43.31991 -4.13143),"POLYGON ((-43.32358 -4.12935, -43.31558 -4.130...",0.225592
2,2010,210010505000003,-43.271484,-4.156843,RURAL,2100105,21,0.2328929503293089,0.85,0.5655806182121972,POINT (-43.27148 -4.15684),"POLYGON ((-43.31558 -4.13098, -43.32358 -4.129...",130.898644
3,2010,210010505000004,-43.243892,-4.300203,RURAL,2100105,21,0.21346950704793635,0.7666666666666667,0.6129912758054763,POINT (-43.24389 -4.30020),"POLYGON ((-43.24540 -4.23016, -43.24366 -4.233...",53.153215
4,2010,210010505000005,-43.331105,-4.230654,RURAL,2100105,21,0.27818798951147716,0.7666666666666667,0.7001609442060085,POINT (-43.33110 -4.23065),"POLYGON ((-43.32746 -4.12979, -43.32786 -4.136...",190.073582


In [ ]:
ala = geototal[geototal['Cod_setor'] == '270010205000001'];
ala.head()

,year,Cod_setor,lat,lon,TIPO,Cod_municipio,Cod_estado,income,longevity,literacy,centroid,geometry,area
45061,2010,270010205000001,-37.940131,-9.266057,URBANO,2700102,27,0.48928633496555013,0.7333333333333333,0.8320838139177239,POINT (-37.94013 -9.26606),"POLYGON ((-37.94456 -9.26639, -37.94241 -9.265...",0.312876


In [ ]:
geototal.to_csv("IDH_2010_teste.csv")